# Anomaly detection
Anomaly detection detect unusual events by comparing it to normal data

## Gaussian distribution
For a number $x$, it's probability, $p(x)$, is determined by normal distribution with a mean value ($\mu$) and a standard deviatioin ($\sigma$), where $\mu$ determines the center of the curve and $\sigma$ determines the spread of the data

The probability curve is given by
$$ p(x) = \frac{1}{\sqrt{2 \pi \sigma ^2}}\exp^{ - \frac{(x - \mu)^2}{2 \sigma ^2} },$$
where $p(x)$ determines the probability of the given feature being "normal"

<img src="https://www.inchcalculator.com/wp-content/uploads/2021/11/normal-distribution-graph.png" width=500>

Note: the total area under the curve always equals 1

## Density estimation
Given a threshold $\epsilon$ and a test data ($x_{test}$), if $p(x_{test}) \geq \epsilon$, the data is considered "normal". If $p(x_{test}) < \epsilon$, the data is considered an anomaly

## Anomaly detection algorithm
Given a training set $\{(\vec x^{(1)}, \vec x^{(2)}, ... \vec x^{(m)})\}$ with $m$ training example, where each training example is a column vector containing $n$ features

To determine if a test data is "normal", we want to first determine how "normal" is each feature of the test example

### Calculating $\mu$ and $\sigma$ for each feature
$$\mu_i = \frac{1}{m} \sum_{j=1}^m x_i^{(j)}$$

$$\sigma_i^2 = \frac{1}{m} \sum_{j=1}^m (x_i^{(j)} - \mu_i)^2$$

$\mu_i$: the mean of $i$th feature

$x_i^{(j)}$: the value of $i$th feature for $j$th training example 

$\sum_{j=1}^m x_i^{(j)}$: the sum of $i$th feature across all training examples

$\sigma_i^2$: the variance $i$th feature


### Vectorzied implementation

$$\vec \mu = \frac{1}{m} \sum_{i=1}^m \vec x^{(i)}$$

$\vec \mu$: a vector with $n$ entries representing the mean of all features

$x^{(i)}$: $i$th training example that contains $n$ features

### Final probability
After calculating the mean and variance for all $n$ features ($\mu_1, \mu_2, ..., \mu_n, \sigma_1^2, \sigma_2^2, ..., \sigma_n^2$), the probability is

$$p(\vec x) = p(x_1, \mu_1, \sigma_1^2) * p(x_2, \mu_2, \sigma_2^2) * ... * p(x_n, \mu_n, \sigma_n^2)$$
$$= \Pi_{j=1}^{n} p(x_j, \mu_j, \sigma_j^2)$$
$$= \Pi_{j=1}^{n} \frac{1}{\sqrt{2 \pi \sigma_j^2}}\exp^{ - \frac{(x_j - \mu_j)^2}{2 \sigma_j^2} }$$

$\vec x$: a given test data

$p(x_j, \mu_j, \sigma_j^2)$: the probability of $j$th feature being "normal"

$\Pi_{j=1}^{n} p(x_j, \mu_j, \sigma_j^2)$: the consecutive multiplication of the probabilty of each feature being "normal"

We can then take $p(\vec x)$ and compare it to the threshold, $\epsilon$

## Selecting thershold $\epsilon$
To select the threshold, we can set multiple $\epsilon$ values and calculate the F1 score using precision and recall. Then, select the $\epsilon$ value with the highest F1 score (highest precision and recall) as the threshold

# Code

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Calculate mean and variance for all features
def estimate_gaussian(X): 
    """
    Args:
        X (ndarray): (m, n) Data matrix
    
    Returns:
        mu (ndarray): (n,) Mean of all features
        var (ndarray): (n,) Variance of all features
    """
    
    # Get number of training examples and features
    m, n = X.shape
    
    # Initialize an array with n entries
    mu = np.zeros(n)
    var = np.zeros(n)
    
    # Calculating mu
    # Looping through each feature
    for j in range(n):
        # Looping through each training examples
        for i in range(m):
            mu[j] += X[i, j]
    mu /= m
    
    # Calculating variance
    for j in range(n):
        for i in range(m):
            var[j] += (X[i, j] - mu[j]) ** 2
    
    var /= m
    
    return mu, var

In [6]:
# Test
X_train = np.array([[15.79025979, 14.9210243 ],
 [13.63961877, 15.32995521],
 [14.86589943, 16.47386514],
 [13.58467605, 13.98930611],
 [13.46404167, 15.63533011],])

mu, var = estimate_gaussian(X_train)              

print("Mean of each feature:", mu)
print("Variance of each feature:", var)

Mean of each feature: [14.26889914 15.26989617]
Variance of each feature: [0.83657963 0.66966256]


In [9]:
# Compute probablity density function
def multivariate_gaussian(X, mu, var):
    """
    Computes the probability 
    density function of the examples X under the multivariate gaussian 
    distribution with parameters mu and var. If var is a matrix, it is
    treated as the covariance matrix. If var is a vector, it is treated
    as the var values of the variances in each dimension (a diagonal
    covariance matrix
    """
    
    k = len(mu)
    
    if var.ndim == 1:
        var = np.diag(var)
        
    X = X - mu
    p = (2* np.pi)**(-k/2) * np.linalg.det(var)**(-0.5) * \
        np.exp(-0.5 * np.sum(np.matmul(X, np.linalg.pinv(var)) * X, axis=1))
    
    return p

In [10]:
# Finds the best threshold to use for selecting outliers based on the results from a validation set (p_val) and the ground truth (y_val)
def select_threshold(y_val, p_val): 
    """
    Args:
        y_val (ndarray): Ground truth on validation set
        p_val (ndarray): Results on validation set
        
    Returns:
        epsilon (float): Threshold chosen 
        F1 (float):      F1 score by choosing epsilon as threshold
    """ 

    best_epsilon = 0
    best_F1 = 0
    F1 = 0
    
    step_size = (max(p_val) - min(p_val)) / 1000
    
    for epsilon in np.arange(min(p_val), max(p_val), step_size):
    
       # prediction is a binary array that tell if each training example is normal or anomalous
        prediction = (p_val < epsilon)
        
        # Calculation for F1 score
        tp = np.sum((y_val == 1) & (prediction == 1))
        fp = np.sum((y_val == 0) & (prediction == 1))
        fn = np.sum((y_val == 1) & (prediction == 0))
        
        prec = tp / (tp + fp)
        rec = tp / (tp + fn)
        
        F1 = 2 * prec * rec / (prec + rec)
        
        # Update F1 if there is a better one
        if F1 > best_F1:
            best_F1 = F1
            best_epsilon = epsilon
        
    return best_epsilon, best_F1

# Real number evaluation
To evaluate how well an anomaly detection algorithm performs, we can assume or collect some labelled data

Steps
1. Seperate the labelled data by only putting normal training examples into the training set and splitting anomalous examples into the validation and testing set
2. Train the algorithm using the training set (only normal traing examples)
3. Check how well the algorithm predict anomalous data using the validation set and fine tuning the algorithm (eg. change the threshold)
4. Evaluate the algorithm with the test set

Note: if the anomalous examples are really rare, the methods to evaluate skewed data can be applied


# Anomaly detection vs Supervised learning
Anomaly detection should be used when the number of anomalous examples are small, there are multiple types of anomalies, and future anomalies may not looks like any given training data

Supervised learning should be used when there is enough normal and anomalous examples and future examples will look very similar to the training data

# Selecting features to use for anomaly detection
Anomaly detection algorithm works well with gaussian features (features that follow the normal distribution). When having non-gaussian features, we can transform them by
* Taking the square root of the feature
* Taking the log of the feature
* Taking $log(x + c)$, where x is the value of the feature and c is an arbitary constant

# Error analysis
Sometimes, an anomalous example can have $p(x)$ similar to normal examples. We can manually inspect the anomaly to determine why it is an anomaly and creating new features based on the given data to help the algorithm detect it